<center><h1>Indra Rusyadi Adiwijaya<br/>20921001<br/>Laporan Kemajuan Riset IX<br/>
ANALISIS REGRESI LOGISTIK MULTINOMIAL</h1><br/>SK6091 Independent Research in Computational Science 1</center>

<center><h1>ANALISIS REGRESI LOGISTIK MULTINOMIAL PADA DATA MONITORING & EVALUASI TENANT BIT BPPT</h1></center>
<hr>

## Load dataset

Dataset yang digunakan adalah dataset yang berasal dari data MONEV tenant Balai Inkubator Teknologi BPPT menggunakan Balanced Scorecard (BSC) yakni metode pengukuran hasil kerja yang digunakan perusahaan atau biasa disebut dengan strategi menajemen. Informasi tiap kolom sebagai berikut:

__Informasi Atribut pada Data Tersebut :__

1. __Nama Tenant__
2. __Bidang Teknologi__ 
3. __Omset__
4. __Jumlah Tenaga Kerja__
5. __Score MONEV__
6. __Predikat MONEV__

In [1]:
import seaborn as sns #Untuk memanggil data 
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report #Untuk mengukur pemodelan klasifikasi (F1 score, Sensitivity, Recall dsb) 
from sklearn.metrics import accuracy_score #Untuk mengukur baik tidaknya model yang digunakan 
from sklearn.model_selection import train_test_split #Split data training dan data test
from sklearn.linear_model import LogisticRegression #Menampung pemodelan regresi logistic

In [2]:
data = pd.read_csv("datasets/bit.csv")
#data= sns.load_dataset("tenant_bit")
data

,Nama.Tenant,Bidang.Teknologi,Omset,Tenaga.Kerja,Score,Predikat
0,CV. Sentra Susu Cipageuran,Pertanian & Tahan Pangan,360000000,5.0,190,FAIR
1,CV Gerbang Organik Nusantara,Pertanian & Tahan Pangan,250000000,3.0,159,FAIR
2,PT. Agro Teknologi Pintar,ICT,230000000,5.0,296,EXCELLENT
3,PT. Mikata Sukses Mandiri,Pertanian & Tahan Pangan,3600000000,5.0,310,EXCELLENT
4,PT. Adhiguna Jaya Laboratorium,Pertanian & Tahan Pangan,500000000,5.0,223,GOOD
...,...,...,...,...,...,...
72,CV. Mabol Store,Pertanian & Tahan Pangan,17880924,10.0,133,DROP OUT
73,PT Bali Lengis Alus,Pertanian & Tahan Pangan,28539126,5.0,93,DROP OUT
74,MAS Food,Pertanian & Tahan Pangan,38806530,10.0,155,FAIR
75,Sentosa Quail Farm,Pertanian & Tahan Pangan,8627720,4.0,184,FAIR


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Nama.Tenant       77 non-null     object 
 1   Bidang.Teknologi  77 non-null     object 
 2   Omset             77 non-null     int64  
 3   Tenaga.Kerja      76 non-null     float64
 4   Score             77 non-null     int64  
 5   Predikat          77 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 3.7+ KB


In [5]:
# Mengubah tipe data Bidang Teknologi & Predikat Kelulusan menjadi category

data['Bidang.Teknologi'] = data['Bidang.Teknologi'].astype('category')
data['Predikat'] = data['Predikat'].astype('category')
#data_obj = data.select_dtypes(include='object').copy()
#data_obj.head()

In [6]:
# Mengecek tipe data kolom Bidang Teknologi

data['Bidang.Teknologi'].dtype

CategoricalDtype(categories=['Energi', 'Farmasi', 'Hankam', 'ICT', 'Kreatif',
                  'Material Maju', 'Pertanian & Tahan Pangan'],
, ordered=False)

In [7]:
# Mengecek tipe data kolom Predikat

data['Predikat'].dtype

CategoricalDtype(categories=['DROP OUT', 'EXCELLENT', 'FAIR', 'GOOD', 'VERY GOOD'], ordered=False)

In [20]:
# Mengaplikasikan Endcoding kolom Bidang Teknologi menggunakan .cat.codes

data['BidTek-encode'] = data['Bidang.Teknologi'].cat.codes
data.head()

,Nama.Tenant,Bidang.Teknologi,Omset,Tenaga.Kerja,Score,Predikat,BidTek-encode
0,CV. Sentra Susu Cipageuran,Pertanian & Tahan Pangan,360000000,5.0,190,FAIR,6
1,CV Gerbang Organik Nusantara,Pertanian & Tahan Pangan,250000000,3.0,159,FAIR,6
2,PT. Agro Teknologi Pintar,ICT,230000000,5.0,296,EXCELLENT,3
3,PT. Mikata Sukses Mandiri,Pertanian & Tahan Pangan,3600000000,5.0,310,EXCELLENT,6
4,PT. Adhiguna Jaya Laboratorium,Pertanian & Tahan Pangan,500000000,5.0,223,GOOD,6


In [21]:
#X=data.iloc[:, :-1] #Semua baris kecuali kolom paling kanan ":-1"
X=data[['BidTek-encode', 'Omset','Tenaga.Kerja']]
#X=data[['Score']]
y=data['Predikat'] #Menagmbil kolom "Predikat"
print("Berikut ini tampilan data X")
print(X.head()) #Variabel Independen (X)
print("")
print("Berikut ini tampilan data Y")
print(y.head()) #Variabel Dependen (Y)

Berikut ini tampilan data X
   BidTek-encode       Omset  Tenaga.Kerja
0              6   360000000           5.0
1              6   250000000           3.0
2              3   230000000           5.0
3              6  3600000000           5.0
4              6   500000000           5.0

Berikut ini tampilan data Y
0         FAIR
1         FAIR
2    EXCELLENT
3    EXCELLENT
4         GOOD
Name: Predikat, dtype: category
Categories (5, object): ['DROP OUT', 'EXCELLENT', 'FAIR', 'GOOD', 'VERY GOOD']


In [18]:
X_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3) #Data train 70% Data Test 30%

In [19]:
#data.replace([np.inf, -np.inf], np.nan, inplace=True)
#data.fillna(999, inplace=True)

model=LogisticRegression(multi_class='auto') #Membentuk model regresi logstik, "auto" supaya python mendeteksi
model.fit(X_train,y_train) #Fit model logistik regresion data train 70%


#data.fillna(df.mean(), inplace=True)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [22]:
print("Berikut ini tampilan data x_test")
print(x_test.head()) 

Berikut ini tampilan data x_test
    BidTek-encode      Omset  Tenaga.Kerja
39              5   44831304          10.0
5               6  300000000           3.0
16              0  400000000           6.0
13              5   32000000           3.0
60              6  386000000           5.0


In [13]:
model.predict_proba(x_test)[0:5] #Prediksi Probabilitas x_test di 5 data pertama

array([[9.15496503e-04, 2.02850040e-50, 9.99084464e-01, 3.92201810e-08,
        3.27991059e-25],
       [1.67232398e-21, 1.89595257e-14, 3.96013412e-08, 9.96900417e-01,
        3.09954304e-03],
       [4.18137475e-20, 4.92032106e-16, 2.63913547e-07, 9.99517815e-01,
        4.81920814e-04],
       [7.18773495e-01, 1.99119708e-65, 2.81226505e-01, 1.28022255e-13,
        1.48892694e-35],
       [3.74142515e-21, 7.61352780e-15, 6.36597337e-08, 9.98052631e-01,
        1.94730576e-03]])

In [23]:
predictions=model.predict(x_test)
predictions

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [24]:
print(classification_report(y_test,predictions))
print("akurasi hasil prediksi adalah sebesar:")
print(accuracy_score(y_test,predictions))

              precision    recall  f1-score   support

    DROP OUT       0.33      0.50      0.40         4
   EXCELLENT       0.17      0.14      0.15         7
        FAIR       0.00      0.00      0.00         4
        GOOD       0.75      0.75      0.75         4
   VERY GOOD       0.00      0.00      0.00         5

    accuracy                           0.25        24
   macro avg       0.25      0.28      0.26        24
weighted avg       0.23      0.25      0.24        24

akurasi hasil prediksi adalah sebesar:
0.25


<hr>

Referensi @ <a href="https://ilmudatapy.com/">ilmudatapy.com</a>